In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pydantic import BaseModel, Field
from typing import List

In [29]:
from package.lib.parse_outline import parse_outline
from package.lib.searxng import SearxngSearchOptions, search_searxng
from package.interfaces.sections import Sections
from broai.experiments.web_scraping import scrape_by_jina_ai
from broai.experiments.cleanup_markdown import clean_up_markdown_link

In [22]:
# Example usage:
outline = """
## Overview of Insurance Fraud  
- what is insurance fraud look like  
- what is the impact on insurance fraud in USA  
- how many types of insurance frauds are  
- how to detect insurance fraud
""".strip()

parsed = parse_outline(outline)
print(parsed)

[{'section': 'Overview of Insurance Fraud', 'questions': [{'question': 'what is insurance fraud look like', 'urls': []}, {'question': 'what is the impact on insurance fraud in USA', 'urls': []}, {'question': 'how many types of insurance frauds are', 'urls': []}, {'question': 'how to detect insurance fraud', 'urls': []}]}]


In [23]:
sections = Sections(sections=parsed)

In [24]:
sections.sections[0].questions[0]

Question(question='what is insurance fraud look like', urls=[])

In [26]:
from package.lib.searxng import SearxngSearchOptions, search_searxng
opts = SearxngSearchOptions(
    engines=["google", "bing", "DuckDuckGo"],
    language="en",
    pageno=1
)
results = search_searxng("what is insurance fraud look like", opts)

In [27]:
for result in results.results:
    print(result.url)
    # print(result.content)

https://gia.org.sg/motor-insurance/22-premium-renewal-of-policy/358-motor-insurance-fraud.html
https://www.budgetdirect.com.sg/car-insurance/articles/how-to-spot-motor-insurance-fraud
https://doi.nebraska.gov/sites/default/files/doc/examples.pdf
https://www.allstate.com/resources/car-insurance/types-of-car-insurance-fraud
https://www.steadfast.com.sg/well-covered/business-insurance/insurance-fraud-what-is-it-and-what-are-its-implications/
https://en.wikipedia.org/wiki/Insurance_fraud
https://content.naic.org/article/consumer-insight-insurance-fraud
https://www.vero.co.nz/vero-voice/understanding-insurance-fraud.html
https://www.scor.com/en/insurance-fraud-it-growing-trend
https://www.iii.org/publications/insurance-handbook/regulatory-and-financial-environment/background-on-insurance-fraud


In [31]:
text = scrape_by_jina_ai(result.url)
cleaned_text = clean_up_markdown_link(text)

In [33]:
print(cleaned_text[:1000])

Title: Background on: Insurance Fraud

URL Source: https://www.iii.org/publications/insurance-handbook/regulatory-and-financial-environment/background-on-insurance-fraud

Published Time: 2014-01-12T17:11:16-05:00

Markdown Content:
The topic
---------

Insurance fraud is a deliberate deception perpetrated against or by an insurance company or agent for financial gain. Fraud may be committed at different points by applicants, policyholders, third-party claimants, or professionals who provide services to claimants. Insurance agents and company employees may also commit insurance fraud. Common frauds include "padding" (inflating claims), misrepresenting facts on an insurance application, submitting claims for injuries or damage that never occurred, and staging accidents.

People who commit insurance fraud include:

*   organized criminals who steal large sums through fraudulent business activities,
*   professionals and technicians who inflate service costs or charge for services not rend

In [34]:
print(text[:1000])

Title: Background on: Insurance Fraud

URL Source: https://www.iii.org/publications/insurance-handbook/regulatory-and-financial-environment/background-on-insurance-fraud

Published Time: 2014-01-12T17:11:16-05:00

Markdown Content:
The topic
---------

Insurance fraud is a deliberate deception perpetrated against or by an insurance company or agent for financial gain. Fraud may be committed at different points by applicants, policyholders, third-party claimants, or professionals who provide services to claimants. Insurance agents and company employees may also commit insurance fraud. Common frauds include "padding" (inflating claims), misrepresenting facts on an insurance application, submitting claims for injuries or damage that never occurred, and staging accidents.

People who commit insurance fraud include:

*   organized criminals who steal large sums through fraudulent business activities,
*   professionals and technicians who inflate service costs or charge for services not rend

In [57]:
len(cleaned_text.split("\n\n"))

1

In [56]:
print(cleaned_text.split("\n\n")[50])

State antifraud legislation
---------------------------


In [59]:
from broai.experiments.chunk import split_overlap
from broai.interface import Context

In [60]:
context = Context(context=cleaned_text, metadata={"source": "abc"})

In [61]:
new_contexts = split_overlap([context])

/tmp/ipykernel_11908/4223221696.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap([context])


In [62]:
len(new_contexts)

10

In [65]:
print(new_contexts[0].context)

Title: Background on: Insurance Fraud

URL Source: https://www.iii.org/publications/insurance-handbook/regulatory-and-financial-environment/background-on-insurance-fraud

Published Time: 2014-01-12T17:11:16-05:00

Markdown Content:
The topic
---------

Insurance fraud is a deliberate deception perpetrated against or by an insurance company or agent for financial gain. Fraud may be committed at different points by applicants, policyholders, third-party claimants, or professionals who provide services to claimants. Insurance agents and company employees may also commit insurance fraud. Common frauds include "padding" (inflating claims), misrepresenting facts on an insurance application, submitting claims for injuries or damage that never occurred, and staging accidents.

People who commit insurance fraud include:

*   organized criminals who steal large sums through fraudulent business activities,
*   professionals and technicians who inflate service costs or charge for services not rend

In [66]:
print(new_contexts[1].context)

who was at fault in a motor vehicle accident. However, in many no-fault states, unscrupulous medical providers, attorneys, and others pad costs associated with legitimate claims – for example, by billing an insurer for a medical procedure never performed.

### Salvage fraud

Another common auto fraud involves vehicles damaged by storm flooding that later appear in used car lots and auction sales. Some states require "salvage only" on the titles of flooded vehicles, usually if the damage amounts to 75 percent of the car's value. Unscrupulous sellers may switch or clone the manufacturers' serial number plates and put them on a flooded but repaired vehicle. Or, they may resell a car with a salvage-only title in a state with more lax title standards. This practice is called "title washing."

Standardized state rules for titling vehicles are necessary to combat salvage fraud. In recent years, some states in hurricane-prone parts of the United States have adopted rules requiring the words "f

In [67]:
from broai.experiments.vector_store import DuckVectorStore
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/home/ec2-user/SageMaker/broai-research-rocket/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_11908/655537899.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 139965.65it/s]


In [70]:
vs = DuckVectorStore(db_name="./rocket.db", table="knowledge", embedding=baai_em)

/tmp/ipykernel_11908/868377394.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  vs = DuckVectorStore(db_name="./rocket.db", table="knowledge", embedding=baai_em)


In [73]:
len(vs.vector_search(search_query="insurance")[0].context.split(" "))

500

In [74]:
100/4

25.0

In [78]:
(1/3)*100

33.33333333333333

In [79]:
int((3/4)*100)

75

In [1]:
from package.lib.web_register import WebRegister, UrlRecord, WebStatus

In [2]:
wr = WebRegister(db_name="./web_list.db")

In [3]:
wr.read_all()['status'].value_counts()

status
done    164
Name: count, dtype: int64

In [4]:
from broai.experiments.vector_store import DuckVectorStore
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()


def get_vectorstore(db_name, table):
    return DuckVectorStore(db_name, table, embedding=baai_em)

/home/ec2-user/SageMaker/broai-research-rocket/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_8136/3455107429.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 194781.92it/s]


In [5]:
vs = get_vectorstore(db_name="./rocket.db", table="knowledge")

/tmp/ipykernel_8136/3455107429.py:7: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  return DuckVectorStore(db_name, table, embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [6]:
vs.read_all().shape

(1872, 7)

# Test API

In [7]:
import requests
base_url = "http://localhost:8000"
response = requests.get(f"{base_url}/")

In [9]:
response.json()

{'message': 'alive'}

In [15]:
headers = {
    "Content-type": "application/json"
}
data = {
    "session_id": "abc123",
    "tone_of_voice": "test tone",
    "outline": "test outline"
}
response = requests.post(f"{base_url}/outline", headers=headers, json=data)

In [16]:
response.json()

{'session_id': 'abc123',
 'tone_of_voice': 'test tone',
 'outline': 'test outline'}

In [36]:
from typing import List, Dict, Any
from pydantic import BaseModel, Field
from uuid import uuid4
from broai.utils import get_timestamp
from datetime import datetime
class SessionRecord(BaseModel):
    id:str = Field(default_factory=lambda: str(uuid4()))
    tone_of_voice:str = Field()
    outline:str = Field()
    retrieved:List[Dict[str, Any]] = Field(default_factory=list)
    summarized:List[Dict[str, Any]] = Field(default_factory=list)
    published:List[Dict[str, Any]] = Field(default_factory=list)
    created_at:datetime = Field(default_factory=get_timestamp)

In [37]:
SessionRecord(tone_of_voice="Test", outline="Test Test")

SessionRecord(id='6fa8757c-6c4f-4101-8217-115a5af656d8', tone_of_voice='Test', outline='Test Test', retrieved=[], summarized=[], published=[], created_at='2025-05-03 22:44:38.976162')

In [38]:
tone_of_voice = "Write an article as the introduction to Insurance Fraud. Make the language easy and simple for the beginners. The article aims to educate a new fraud investigator."

outline = """\
## Overview of Insurance Fraud  
- what is insurance fraud look like  
- what is the impact on insurance fraud in USA  
- how many types of insurance frauds are  
- how to detect insurance fraud  
""".strip()

In [39]:
session = SessionRecord(id="test", tone_of_voice=tone_of_voice, outline=outline)

In [40]:
session

SessionRecord(id='test', tone_of_voice='Write an article as the introduction to Insurance Fraud. Make the language easy and simple for the beginners. The article aims to educate a new fraud investigator.', outline='## Overview of Insurance Fraud  \n- what is insurance fraud look like  \n- what is the impact on insurance fraud in USA  \n- how many types of insurance frauds are  \n- how to detect insurance fraud', retrieved=[], summarized=[], published=[], created_at='2025-05-03 22:44:40.217464')

In [45]:
headers = {
    "Content-type": "application/json"
}

response = requests.post(f"{base_url}/start-session", headers=headers, json=session.model_dump())

In [46]:
response.json()

{'response': [{'section': 'Overview of Insurance Fraud',
   'questions': ['what is insurance fraud look like',
    'what is the impact on insurance fraud in USA',
    'how many types of insurance frauds are',
    'how to detect insurance fraud']}]}